# Assignment 4: Quantum Channel Classification
We reuse the Week 3 classifier to tag noisy channels quickly during calibration checks.

This week we focus on applying the saved classifier and showing that the workflow runs end to end.

**Task plan**
1. Explain why quick channel labels matter and list the workflow you will follow.
2. Load the helper code, pull in the trained estimator, and note any backup plan.
3. Rebuild the feature mapper from Kraus operators to Choi vectors.
4. Test the model on a small batch of synthetic channels and record observations.

## Background notes
- A quantum channel $\mathcal{E}$ is a completely positive, trace-preserving map. We write $\mathcal{E}(\rho) = \sum_k K_k \rho K_k^\dagger$ with $\sum_k K_k^\dagger K_k = I$ so that each Kraus operator $K_k$ captures one noise branch.
- The Choi matrix $C_{\mathcal{E}} = (\mathcal{E} \otimes I)(|\Phi^+\rangle\langle\Phi^+|)$ models how the channel acts on half of an entangled pair. Flattening its real and imaginary parts gives a steady feature vector.
- Classification is lighter than full tomography. We only emit labels like depolarising or amplitude damping, which keeps the calibration loop fast.

## Task 1 · Environment check
- Confirm qiskit, numpy, pandas, joblib, and scikit-learn import without errors.
- If anything is missing, run the pip cell below and log the command in your notes.
- Once the imports work, move to Task 2.

In [21]:
# Task 1: Environment check

try:
    import numpy
    import pandas
    import joblib
    import sklearn
    import qiskit
    print("All required packages imported successfully")
except ImportError as e:
    print("Missing package:", e)


All required packages imported successfully


In [22]:
# Install prerequisites if the kernel is missing a package.
# !pip install qiskit scikit-learn joblib pandas
!pip install qiskit

## Task 2 · Import helper modules
- Run the cell below to pull in numpy, joblib, qiskit, pandas, and os.
- Keep the imports in one place so later tasks stay consistent.

In [23]:
import numpy as np
import joblib
from sklearn.dummy import DummyClassifier
from qiskit.quantum_info import Kraus, Choi
import pandas as pd
import os


## Task 3 · Build a calibration-time classifier
- Implement a lightweight classifier that maps Choi features to channel labels without relying on saved artefacts.
- Keep the training code inside the provided function so reviewers can see your modelling choices.
- You may reuse utilities from earlier assignments (data loaders, feature encoders) as long as they are imported inside the function.

In [24]:
import numpy as np

# Kraus Channels


def depolarizing_kraus(p):
    I = np.eye(2)
    X = np.array([[0, 1],
                  [1, 0]])
    Y = np.array([[0, -1j],
                  [1j, 0]])
    Z = np.array([[1, 0],
                  [0, -1]])

    K0 = np.sqrt(1 - p) * I
    K1 = np.sqrt(p / 3) * X
    K2 = np.sqrt(p / 3) * Y
    K3 = np.sqrt(p / 3) * Z

    return [K0, K1, K2, K3]


def amplitude_damping_kraus(gamma):
    K0 = np.array([[1, 0],
                   [0, np.sqrt(1 - gamma)]])
    K1 = np.array([[0, np.sqrt(gamma)],
                   [0, 0]])

    return [K0, K1]

# Choi Conversion

def kraus_to_choi(kraus_ops):
    d = kraus_ops[0].shape[0]
    choi = np.zeros((d*d, d*d), dtype=complex)

    for K in kraus_ops:
        choi += np.kron(K, np.conj(K))

    return choi


def channel_to_feature(kraus_ops):
    choi = kraus_to_choi(kraus_ops)

    real_part = np.real(choi).flatten()
    imag_part = np.imag(choi).flatten()

    return np.concatenate([real_part, imag_part])


In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

def build_channel_classifier():
    """
    Train a classifier to distinguish:
        - depolarizing
        - amplitude_damping
    """

    # Generate synthetic dataset
    ps = np.linspace(0.01, 0.9, 30)
    gammas = np.linspace(0.01, 0.9, 30)

    X_data = []
    y_data = []

    # Depolarizing the samples
    for p in ps:
        ch = depolarizing_kraus(p)
        feat = channel_to_feature(ch)
        X_data.append(feat)
        y_data.append("depolarizing")

    # Amplitude damping the samples
    for g in gammas:
        ch = amplitude_damping_kraus(g)
        feat = channel_to_feature(ch)
        X_data.append(feat)
        y_data.append("amplitude_damping")

    X_data = np.array(X_data)
    y_data = np.array(y_data)

    # Train/validation split
    X_train, X_val, y_train, y_val = train_test_split(
        X_data, y_data, test_size=0.2, random_state=42
    )

    # Train model
    model = LogisticRegression(max_iter=2000)
    model.fit(X_train, y_train)

    # Evaluate
    y_pred = model.predict(X_val)
    print("Validation Accuracy:", accuracy_score(y_val, y_pred))
    print(classification_report(y_val, y_pred))

    return model

model = build_channel_classifier()


Validation Accuracy: 0.9166666666666666
                   precision    recall  f1-score   support

amplitude_damping       0.89      1.00      0.94         8
     depolarizing       1.00      0.75      0.86         4

         accuracy                           0.92        12
        macro avg       0.94      0.88      0.90        12
     weighted avg       0.93      0.92      0.91        12



## Task 4 · Build channel features
- Regenerate the Kraus operators you used during training or adapt them for this demo.
- Ensure `channel_to_feature` outputs the same ordering the model expects (real part first, imaginary part second).

In [26]:
I = np.eye(2, dtype=complex)
X = np.array([[0, 1], [1, 0]], dtype=complex)
Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
Z = np.array([[1, 0], [0, -1]], dtype=complex)

def depolarizing_kraus(p):
    k0 = np.sqrt(1 - p) * I
    k1 = np.sqrt(p/3) * X
    k2 = np.sqrt(p/3) * Y
    k3 = np.sqrt(p/3) * Z
    return Kraus([k0, k1, k2, k3])

def amplitude_damping_kraus(gamma):
    k0 = np.array([[1, 0], [0, np.sqrt(1-gamma)]], dtype=complex)
    k1 = np.array([[0, np.sqrt(gamma)], [0, 0]], dtype=complex)
    return Kraus([k0, k1])

def channel_to_feature(channel):
    choi = Choi(channel).data
    feat = np.concatenate([choi.real.flatten(), choi.imag.flatten()])
    return feat


## Task 5 · Classify sample channels
- Build a small list of synthetic channels, convert them with `channel_to_feature`, and stack the results in `X`.
- Use the loaded model to predict labels and review the DataFrame for any surprising cases.

In [27]:
channels = [
    ('depolarizing_p0.1', depolarizing_kraus(0.1)),
    ('depolarizing_p0.5', depolarizing_kraus(0.5)),
    ('amp_damp_0.1', amplitude_damping_kraus(0.1)),
    ('amp_damp_0.5', amplitude_damping_kraus(0.5)),
]

features = []
names = []
for name, ch in channels:
    f = channel_to_feature(ch)
    names.append(name)
    features.append(f)
X = np.vstack(features)

preds = model.predict(X)
df = pd.DataFrame({'channel': names, 'prediction': preds})
df

,channel,prediction
0,depolarizing_p0.1,depolarizing
1,depolarizing_p0.5,depolarizing
2,amp_damp_0.1,depolarizing
3,amp_damp_0.5,depolarizing


### Submission checklist
- Update `model_path` with the actual artifact you trained in Assignment 3 and note the load result.
- Mention any feature changes you make so the classifier stays compatible with production runs.
- Save this notebook with outputs after running Tasks 1–5 and add a short reflection in your report.